Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [13]:
from random import choices, randint
import random
import math


import lab9_lib

In [14]:
LOCI = 1000
MUT_OR_REC = 0.5
MAX_GENS = 8000
POP_SIZE = 40
OFF_SIZE = 20

### Parent Selection
Tournament selection between the best 2 parents and 2 random parents with weighted probability of victory based on fitness
Winners can be either both best parents, a combination of a best parent and a random one or both random selections

In [15]:
def parent_selection(population):
    pool = []   
    pool1 = [population[0],population[1]]
    pool2 = choices(population, k=2)
    pool2.sort(key=lambda x: x[1], reverse=True)

    if random.random() > pool1[0][1] / (pool1[0][1]+pool2[0][1]):
        pool.append(pool1[0])
    else:
        pool.append(pool2[0])
    
    if random.random() > pool1[1][1] / (pool1[1][1]+pool2[1][1]):
        pool.append(pool1[1])
    else:
        pool.append(pool2[1])

    return pool




### Mutate Function
Mutate a number between 0 and 30 of randomly sampled genes

In [16]:
def mutate(parent):
    slice_size = randint(0, 30)
    offspring = parent.copy()

    mutate_indexes = random.sample(range(0, LOCI), slice_size)

    for i in mutate_indexes:
        offspring[i] = 1 - offspring[i]

    return offspring

### Recombination
Applied with random slice size

In [17]:
def recombine(p1, p2):
    slice_size = randint(0, LOCI - 1)

    off1 = p1[:slice_size] + p2[slice_size:]
    off2 = p2[:slice_size] + p1[slice_size:]

    return ( off1, off2 )

### Survival Selection

In [18]:
def survival_selection(population):
    population.sort(key=lambda x: x[1], reverse=True)
    return population[:POP_SIZE]


### Offspring Generator
Chooses between mutation or recombination with 50:50 probability

In [19]:
def generate_offspring(population):
    individuals = []
    for _ in range(OFF_SIZE):
        
        parents = parent_selection(population)

        if random.random() <= MUT_OR_REC:
            offsprings = (mutate(parents[0][0]), mutate(parents[1][0]))
        else: 
            offsprings = recombine(parents[0][0], parents[1][0])

        individuals.append(offsprings[0])
        individuals.append(offsprings[1])

    return individuals

### Genetic Algorithm

In [20]:
def genetic_algorithm(instances):
    fitness = lab9_lib.make_problem(instances)

    population = []
    for _ in range(POP_SIZE):
        individual = [random.randint(0, 1) for _ in range(LOCI)]
        population.append((individual, fitness(individual)))
    
    best_individual = population[0]

    for _ in range(MAX_GENS):

        if math.isclose(1, population[0][1]):
            break
        
        offsprings = generate_offspring(population)
        population += [(individual, fitness(individual)) for individual in offsprings]
        population = survival_selection(population)

        best_individual = population[0]
    
    print(f"Best individual's fitness: {best_individual[1]:.2%}")
    print("Fitness calls: " + str(fitness.calls))

### Instances: 1

In [21]:
genetic_algorithm(1)

Best individual's fitness: 100.00%
Fitness calls: 281840


### Instances: 2

In [22]:
genetic_algorithm(2)

Best individual's fitness: 90.40%
Fitness calls: 320040


### Instances: 5

In [23]:
genetic_algorithm(5)

Best individual's fitness: 50.37%
Fitness calls: 320040


### Instances: 10

In [24]:
genetic_algorithm(10)

Best individual's fitness: 32.77%
Fitness calls: 320040
